**INSTALL UNSLOTH & DEPENDENCIES**

In [1]:
!pip install -q unsloth
!pip install -q datasets accelerate bitsandbytes trl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 129.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 129.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**IMPORTANT LIBRARIES**

In [2]:

import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


LOAD BASED MODEL(4-BIT QLORA)

In [3]:
model_name = "unsloth/llama-3-8b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    load_in_4bit = True,
    dtype = None,
)


==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

ADD LORA ADAPTER(PEFT)

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,                     # LoRA rank
    lora_alpha = 32,
    lora_dropout = 0.05,
    target_modules = [
        "q_proj", "k_proj", "v_proj",
        "o_proj", "gate_proj", "up_proj", "down_proj"
    ],
    use_gradient_checkpointing = "unsloth",
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.2.1 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


**MEDICAL Q&A DATASET**

In [5]:

dataset = load_dataset("pubmed_qa", "pqa_labeled", split="train[:2000]")

README.md: 0.00B [00:00, ?B/s]

pqa_labeled/train-00000-of-00001.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

**FORMAT DATASET**

In [6]:
def formatting_func(example):
    return f"""### Question:
{example['question']}

### Answer:
{example['long_answer']}"""

dataset = dataset.map(lambda x: {
    "text": formatting_func(x)
})

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

**SETUP QLORA TRAINING**

In [14]:

training_args = TrainingArguments(
    output_dir = "./medical_llama3",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    learning_rate = 2e-4,
    num_train_epochs = 2,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_8bit",
    save_strategy = "epoch",
    report_to = "none"
)

**START FINE_TUNING**

In [15]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = training_args,
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 2 | Total steps = 250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
10,2.282400
20,1.971600
30,2.078500
40,2.066800
50,2.078000
60,2.080600
70,1.957300
80,2.019200
90,2.036300
100,1.957600


TrainOutput(global_step=250, training_loss=1.8265573272705078, metrics={'train_runtime': 890.054, 'train_samples_per_second': 2.247, 'train_steps_per_second': 0.281, 'total_flos': 8047828790722560.0, 'train_loss': 1.8265573272705078, 'epoch': 2.0})

**MONITOR GPU MEMORY(GOOGLE_COLAB)**

In [10]:

torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   5624 MiB |   6798 MiB |  14437 MiB |   8812 MiB |\n|       from large pool |   5358 MiB |   6798 MiB |  14170 MiB |   8812 MiB |\n|       from small pool |    266 MiB |    266 MiB |    267 MiB |      0 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   5624 MiB |   6798 MiB |  14437 MiB |   8812 MiB |\n|       from large pool |   5358 MiB |   6798 MiB |

**SAVE FINE_TUNE** **ADAPTER**

In [11]:
model.save_pretrained("medical_lora_adapter")
tokenizer.save_pretrained("medical_lora_adapter")


('medical_lora_adapter/tokenizer_config.json',
 'medical_lora_adapter/special_tokens_map.json',
 'medical_lora_adapter/tokenizer.json')

**Test MODEL ON NEW MEDICAL** **QUEST**

In [13]:
FastLanguageModel.for_inference(model)

prompt = """### Question:
What are the symptoms of fever?

### Answer:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 200,
    do_sample = True,
    temperature = 0.7,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Question:
What are the symptoms of fever?

### Answer:
Fever is a rise in body temperature. The body temperature of a healthy person is around 36.4 degrees Celsius. Fever is considered to be above 37.8 degrees Celsius. Fever is not a disease. It is a symptom of some disease. Fever is caused by a change in the body's thermostat. This thermostat is called hypothalamus. It is located in the brain. Fever can be caused by infections, injuries, and even stress. When we have fever, our body is trying to fight the infection. Infections like malaria, flu, and typhoid cause fever. Fever is also caused by injuries. Injuries like fractures, burns, and sunburns can cause fever. Even stress can cause fever. Fever can be caused by mental stress. Mental stress can cause fever. Stress can also cause other symptoms like headache, body ache, and nausea. Fever can be caused by a rise in the body temperature. When the body temperature rises, the body temperature rises.
